*Copyright 2024 The Penzai Authors.*

*Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License.
You may obtain a copy of the License at*

> http://www.apache.org/licenses/LICENSE-2.0

*Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or  implied.
See the License for the specific language governing permissions and
limitations under the License.*

---

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/google-deepmind/penzai/blob/main/notebooks/treescope_prettyprinting.ipynb) [![Open in Kaggle](https://kaggle.com/static/images/open-in-kaggle.svg)](https://kaggle.com/kernels/welcome?src=https://github.com/google-deepmind/penzai/blob/main/notebooks/treescope_prettyprinting.ipynb)

# Pretty-Printing With Treescope

Treescope is Penzai's interactive, color-coded HTML pretty-printer, designed for use in IPython notebooks. It's designed to show you the structure of any model or tree of arrays, and is especially suited to looking at nested data structures.

As its name suggests, treescope is specifically focused on inspecting *treelike data*, represented as nodes (Python objects) that contain collections of child nodes (other Python objects). This is a good fit for JAX, since JAX's PyTrees are already tree shaped, and also since JAX works with immutable data structures. It's also a close match to the behavior of the ordinary Python `repr`, which produces a flat source-code-like view of an object and its contents. (Treescope has limited support for more general Python reference graphs and cyclic references as well, but it always renders them in a tree-like form.)

This notebook shows how to use the treescope pretty-printer to look at nested data structures and Penzai models.

## Setup

Let's start by setting up the environment.

### Imports

To run this notebook, you need a Python environment with `penzai` and its dependencies installed.

In Colab or Kaggle, you can install it using the following command:

In [1]:
try:
  import penzai
except ImportError:
  !pip install penzai[notebook]

In [2]:
from __future__ import annotations

import typing
from typing import Any

import dataclasses

import jax
import jax.numpy as jnp
import numpy as np

import IPython

/usr/local/lib/python3.10/dist-packages/jax/_src/cloud_tpu_init.py:73: UserWarning: JAX_USE_PJRT_C_API_ON_TPU no longer has an effect (the new TPU runtime is always enabled now). Unset the environment variable to disable this warning.
  warnings.warn(


In [3]:
import penzai
from penzai import pz

## Overview of Treescope

How does treescope work in practice? Here's an example. Ordinarily, if you try to inspect a nested object containing NDArrays, you get something pretty hard to interpret. For instance, here's a dictionary of parameters rendered using the default IPython pretty-printer:

In [4]:
from penzai.example_models import simple_mlp

mlp = pz.nn.initialize_parameters(
    simple_mlp.MLP.from_config([64, 1264, 128, 128, 64]),
    jax.random.key(1)
)

param_dict = {
    param.name: param.value.data_array
    for param in pz.select(mlp).at_instances_of(pz.nn.Parameter).get_sequence()
}

In [5]:
param_dict

{'Affine_0.Linear.weights': Array([[-0.00339624,  0.02507961,  0.01189755, ...,  0.01883914,
          0.02966279, -0.06463778],
        [ 0.01462419,  0.0251148 ,  0.06374538, ..., -0.03424513,
          0.02044581,  0.04861176],
        [-0.04775642,  0.05381447,  0.01268778, ..., -0.06508733,
          0.00383717, -0.05884858],
        ...,
        [ 0.00079055, -0.04815863,  0.05747943, ..., -0.05628779,
          0.0662893 , -0.01451097],
        [-0.03046485, -0.0284288 , -0.01836847, ..., -0.00243584,
         -0.04748166,  0.01529051],
        [ 0.02892755,  0.05359346, -0.03406196, ...,  0.02147059,
         -0.04189092,  0.01966898]], dtype=float32),
 'Affine_0.AddBias.bias': Array([0., 0., 0., ..., 0., 0., 0.], dtype=float32),
 'Affine_1.Linear.weights': Array([[ 0.0366285 ,  0.05907176, -0.03992373, ...,  0.05467535,
          0.05324409,  0.04470791],
        [-0.01096875,  0.00705392,  0.02313012, ...,  0.05155263,
         -0.03606869, -0.02031791],
        [-0.01198231,

Here's what it looks like if you print it out using the built-in Python `repr`:

In [6]:
print(repr(param_dict))

{'Affine_0.Linear.weights': Array([[-0.00339624,  0.02507961,  0.01189755, ...,  0.01883914,
         0.02966279, -0.06463778],
       [ 0.01462419,  0.0251148 ,  0.06374538, ..., -0.03424513,
         0.02044581,  0.04861176],
       [-0.04775642,  0.05381447,  0.01268778, ..., -0.06508733,
         0.00383717, -0.05884858],
       ...,
       [ 0.00079055, -0.04815863,  0.05747943, ..., -0.05628779,
         0.0662893 , -0.01451097],
       [-0.03046485, -0.0284288 , -0.01836847, ..., -0.00243584,
        -0.04748166,  0.01529051],
       [ 0.02892755,  0.05359346, -0.03406196, ...,  0.02147059,
        -0.04189092,  0.01966898]], dtype=float32), 'Affine_0.AddBias.bias': Array([0., 0., 0., ..., 0., 0., 0.], dtype=float32), 'Affine_1.Linear.weights': Array([[ 0.0366285 ,  0.05907176, -0.03992373, ...,  0.05467535,
         0.05324409,  0.04470791],
       [-0.01096875,  0.00705392,  0.02313012, ...,  0.05155263,
        -0.03606869, -0.02031791],
       [-0.01198231,  0.01880559, -0.0

And here's how it looks in treescope, which is defined in `penzai.treescope` and aliased to `pz.ts` for easier use:

In [7]:
with pz.ts.active_autovisualizer.set_scoped(pz.ts.ArrayAutovisualizer()):
  pz.ts.display(param_dict)

Treescope renders this object as a syntax-highlighted, color-coded structure that can be interactively folded and unfolded.

(Try clicking any `▶` marker to expand a level of the tree, or any `▼` marker to collapse a level.)

In fact, you can even look at the whole model this way, and get a color-coded view of all the parts of your model:

In [8]:
with pz.ts.active_autovisualizer.set_scoped(pz.ts.ArrayAutovisualizer()):
  pz.ts.display(mlp)

Let's register treescope as the default pretty-printer for IPython. This is the recommended way to use treescope in an interactive setting. Treescope is designed to be a drop-in replacement for the ordinary IPython pretty-printer, so you should be able to start using it right away.

In [9]:
pz.ts.register_as_default()

### Foldable and unfoldable nested objects




Treescope lets you expand and collapse any level of your tree, so you can look at the parts you care about. In treescope, you can collapse or expand any object that would render as multiple lines (even if treescope doesn't recognize the type!)

In [10]:
import dataclasses

@dataclasses.dataclass
class MyDataclass:
  a: Any
  b: Any
  c: Any

class TheZenOfPython:
  def __repr__(self):
    return "<The Zen of Python:\nBeautiful is better than ugly.\nExplicit is better than implicit.\nSimple is better than complex.\nComplex is better than complicated.\nFlat is better than nested.\nSparse is better than dense.\nReadability counts.\nSpecial cases aren't special enough to break the rules.\nAlthough practicality beats purity.\nErrors should never pass silently.\nUnless explicitly silenced.\nIn the face of ambiguity, refuse the temptation to guess.\nThere should be one-- and preferably only one --obvious way to do it.\nAlthough that way may not be obvious at first unless you're Dutch.\nNow is better than never.\nAlthough never is often better than *right* now.\nIf the implementation is hard to explain, it's a bad idea.\nIf the implementation is easy to explain, it may be a good idea.\nNamespaces are one honking great idea -- let's do more of those!>"

In [11]:
[
    MyDataclass('a' * i, 'b' * i, ('cccc\n') * i)
    for i in range(10)
] + [
    MyDataclass(TheZenOfPython(), TheZenOfPython(), TheZenOfPython())
]

[MyDataclass(a='', b='', c=''),
 MyDataclass(a='a', b='b', c='cccc\n'),
 MyDataclass(a='aa', b='bb', c='cccc\ncccc\n'),
 MyDataclass(a='aaa', b='bbb', c='cccc\ncccc\ncccc\n'),
 MyDataclass(a='aaaa', b='bbbb', c='cccc\ncccc\ncccc\ncccc\n'),
 MyDataclass(a='aaaaa', b='bbbbb', c='cccc\ncccc\ncccc\ncccc\ncccc\n'),
 MyDataclass(a='aaaaaa', b='bbbbbb', c='cccc\ncccc\ncccc\ncccc\ncccc\ncccc\n'),
 MyDataclass(a='aaaaaaa', b='bbbbbbb', c='cccc\ncccc\ncccc\ncccc\ncccc\ncccc\ncccc\n'),
 MyDataclass(a='aaaaaaaa', b='bbbbbbbb', c='cccc\ncccc\ncccc\ncccc\ncccc\ncccc\ncccc\ncccc\n'),
 MyDataclass(a='aaaaaaaaa', b='bbbbbbbbb', c='cccc\ncccc\ncccc\ncccc\ncccc\ncccc\ncccc\ncccc\ncccc\n'),
 MyDataclass(a=<The Zen of Python:
 Beautiful is better than ugly.
 Explicit is better than implicit.
 Simple is better than complex.
 Complex is better than complicated.
 Flat is better than nested.
 Sparse is better than dense.
 Readability counts.
 Special cases aren't special enough to break the rules.
 Although practicality beats purity.
 Errors should never pass silently.
 Unless explicitly silenced.
 In the face of ambiguity, refuse the temptation to guess.
 There should be one-- and preferably only one --obvious way to do it.
 Although that way may not be obvious at first unless you're Dutch.
 Now is better than never.
 Although never is often better than *right* now.
 If the implementation is hard to explain, it's a bad idea.
 If the implementation is easy to explain, it may be a good idea.
 Namespaces are one honking great idea -- let's do more of those!>, b=<The Zen of Python:
 Beautiful is better than ugly.
 Explicit is better than implicit.
 Simple is better than complex.
 Complex is better than complicated.
 Flat is better than nested.
 Sparse is better than dense.
 Readability counts.
 Special cases aren't special enough to break the rules.
 Although practicality beats purity.
 Errors should never pass silently.
 Unless explicitly silenced.
 In the face of ambiguity, refuse the temptation to guess.
 There should be one-- and preferably only one --obvious way to do it.
 Although that way may not be obvious at first unless you're Dutch.
 Now is better than never.
 Although never is often better than *right* now.
 If the implementation is hard to explain, it's a bad idea.
 If the implementation is easy to explain, it may be a good idea.
 Namespaces are one honking great idea -- let's do more of those!>, c=<The Zen of Python:
 Beautiful is better than ugly.
 Explicit is better than implicit.
 Simple is better than complex.
 Complex is better than complicated.
 Flat is better than nested.
 Sparse is better than dense.
 Readability counts.
 Special cases aren't special enough to break the rules.
 Although practicality beats purity.
 Errors should never pass silently.
 Unless explicitly silenced.
 In the face of ambiguity, refuse the temptation to guess.
 There should be one-- and preferably only one --obvious way to do it.
 Although that way may not be obvious at first unless you're Dutch.
 Now is better than never.
 Although never is often better than *right* now.
 If the implementation is hard to explain, it's a bad idea.
 If the implementation is easy to explain, it may be a good idea.
 Namespaces are one honking great idea -- let's do more of those!>)]

### Copyable key paths

Want to pull out an object deep inside a tree? You can click the  icon next to any subtree to copy a function that accesses that subtree, as a Python source-code lambda expression. You can then paste it into a code cell and pass the original parameter dict to pull out the subtree you wanted.

Try it on one of the parameters of the Penzai model below! (If you run this notebook yourself, you should be able to copy paths with one click. If you are viewing this notebook on Colab without running it, you'll need to click and then copy the path manually due to Colab's security restrictions.)

In [12]:
mlp

MLP( # Sequential
  sublayers=[
    Affine( # Sequential
      sublayers=[Linear(weights=Parameter(value=<NamedArray float32(| features:64, features_out:1264) ≈-0.00015 ±0.039 [≥-0.067, ≤0.067] nonzero:80_896 (wrapping jax.Array)>, name='Affine_0.Linear.weights'), in_axis_names=('features',), out_axis_names=('features_out',)), RenameAxes(old=('features_out',), new=('features',)), AddBias(bias=Parameter(value=<NamedArray float32(| features:1264) ≈0.0 ±0.0 [≥0.0, ≤0.0] zero:1_264 (wrapping jax.Array)>, name='Affine_0.AddBias.bias'), new_axis_names=())],
    ),
    Elementwise(fn=jax.nn.relu),
    Affine( # Sequential
      sublayers=[Linear(weights=Parameter(value=<NamedArray float32(| features:1264, features_out:128) ≈7.8e-05 ±0.038 [≥-0.066, ≤0.066] nonzero:161_792 (wrapping jax.Array)>, name='Affine_1.Linear.weights'), in_axis_names=('features',), out_axis_names=('features_out',)), RenameAxes(old=('features_out',), new=('features',)), AddBias(bias=Parameter(value=<NamedArray float32(| features:128) ≈0.0 ±0.0 [≥0.0, ≤0.0] zero:128 (wrapping jax.Array)>, name='Affine_1.AddBias.bias'), new_axis_names=())],
    ),
    Elementwise(fn=jax.nn.relu),
    Affine( # Sequential
      sublayers=[Linear(weights=Parameter(value=<NamedArray float32(| features:128, features_out:128) ≈-0.0011 ±0.089 [≥-0.15, ≤0.15] nonzero:16_384 (wrapping jax.Array)>, name='Affine_2.Linear.weights'), in_axis_names=('features',), out_axis_names=('features_out',)), RenameAxes(old=('features_out',), new=('features',)), AddBias(bias=Parameter(value=<NamedArray float32(| features:128) ≈0.0 ±0.0 [≥0.0, ≤0.0] zero:128 (wrapping jax.Array)>, name='Affine_2.AddBias.bias'), new_axis_names=())],
    ),
    Elementwise(fn=jax.nn.relu),
    Affine( # Sequential
      sublayers=[Linear(weights=Parameter(value=<NamedArray float32(| features:128, features_out:64) ≈0.00012 ±0.1 [≥-0.18, ≤0.18] nonzero:8_192 (wrapping jax.Array)>, name='Affine_3.Linear.weights'), in_axis_names=('features',), out_axis_names=('features_out',)), RenameAxes(old=('features_out',), new=('features',)), AddBias(bias=Parameter(value=<NamedArray float32(| features:64) ≈0.0 ±0.0 [≥0.0, ≤0.0] zero:64 (wrapping jax.Array)>, name='Affine_3.AddBias.bias'), new_axis_names=())],
    ),
  ],
)

In [13]:
# for example
(lambda root: root.sublayers[6].sublayers[0].weights.value)(mlp)

<NamedArray float32(| features:128, features_out:64) ≈0.00012 ±0.1 [≥-0.18, ≤0.18] nonzero:8_192 (wrapping jax.Array)>

### Structural color-coding for `pz.Struct` and `pz.Layer`

Penzai's base class for PyTree dataclasses includes customizable hooks for treescope rendering. In particular, neural network layers are color-coded by type, so you can see at a glance which parts of your model object are the same type.

By default, any block subclass that defines `__call__` gets a randomly-selected color based on the type name:

In [14]:
@pz.pytree_dataclass
class MyShiftLayer(pz.Layer):
  shift: float

  def __call__(self, value):
    return value + self.shift

@pz.pytree_dataclass
class MyNoOpLayer(pz.Layer):
  def __call__(self, value):
    return value

In [15]:
pz.nn.Sequential([
    MyShiftLayer(1),
    MyShiftLayer(2),
    MyNoOpLayer(),
    MyShiftLayer(3),
])

Sequential(
  sublayers=[
    MyShiftLayer(shift=1),
    MyShiftLayer(shift=2),
    MyNoOpLayer(),
    MyShiftLayer(shift=3),
  ],
)

But you can customize the color by overriding `treescope_color`:

In [16]:
@pz.pytree_dataclass
class MyFancyShiftLayer(pz.Layer):
  name: str = dataclasses.field(metadata={"pytree_node": False})
  shift: float

  def __call__(self, value):
    return value + self.shift

  def treescope_color(self):
    return pz.color_from_string(str(self.name))

In [17]:
pz.nn.Sequential([
    MyFancyShiftLayer("foo", 1),
    MyFancyShiftLayer("bar", 2),
    MyNoOpLayer(),
    MyFancyShiftLayer("foo", 3),
])

Sequential(
  sublayers=[
    MyFancyShiftLayer(name='foo', shift=1),
    MyFancyShiftLayer(name='bar', shift=2),
    MyNoOpLayer(),
    MyFancyShiftLayer(name='foo', shift=3),
  ],
)

In [18]:
@pz.pytree_dataclass
class MyObject(pz.Struct):
  a: float
  b: float

  def treescope_color(self):
    return "cyan"

MyObject(a=1.0, b=2.0)

MyObject(a=1.0, b=2.0)

This is used throughout Penzai to provide color-coded representations that emphasize the behavior of individual parts of complex models.

(There are also some Penzai-specific renderers for understanding dataflow penzai models in particular, which are described in the other tutorial notebooks.)

### Copyable code and roundtrip mode

Documentation for Python's `repr` says:

> For many types, this function makes an attempt to return a string that would yield an object with the same value when passed to eval(); otherwise, the representation is a string enclosed in angle brackets that contains the name of the type of the object together with additional information often including the name and address of the object

Treescope follows this principle for everything it renders. Almost all of the output of treescope is valid Python syntax, and any extra annotations are either hidden from selection or represented as Python comments.

For instance, we'll show again the example MLP model:

In [19]:
mlp

MLP( # Sequential
  sublayers=[
    Affine( # Sequential
      sublayers=[Linear(weights=Parameter(value=<NamedArray float32(| features:64, features_out:1264) ≈-0.00015 ±0.039 [≥-0.067, ≤0.067] nonzero:80_896 (wrapping jax.Array)>, name='Affine_0.Linear.weights'), in_axis_names=('features',), out_axis_names=('features_out',)), RenameAxes(old=('features_out',), new=('features',)), AddBias(bias=Parameter(value=<NamedArray float32(| features:1264) ≈0.0 ±0.0 [≥0.0, ≤0.0] zero:1_264 (wrapping jax.Array)>, name='Affine_0.AddBias.bias'), new_axis_names=())],
    ),
    Elementwise(fn=jax.nn.relu),
    Affine( # Sequential
      sublayers=[Linear(weights=Parameter(value=<NamedArray float32(| features:1264, features_out:128) ≈7.8e-05 ±0.038 [≥-0.066, ≤0.066] nonzero:161_792 (wrapping jax.Array)>, name='Affine_1.Linear.weights'), in_axis_names=('features',), out_axis_names=('features_out',)), RenameAxes(old=('features_out',), new=('features',)), AddBias(bias=Parameter(value=<NamedArray float32(| features:128) ≈0.0 ±0.0 [≥0.0, ≤0.0] zero:128 (wrapping jax.Array)>, name='Affine_1.AddBias.bias'), new_axis_names=())],
    ),
    Elementwise(fn=jax.nn.relu),
    Affine( # Sequential
      sublayers=[Linear(weights=Parameter(value=<NamedArray float32(| features:128, features_out:128) ≈-0.0011 ±0.089 [≥-0.15, ≤0.15] nonzero:16_384 (wrapping jax.Array)>, name='Affine_2.Linear.weights'), in_axis_names=('features',), out_axis_names=('features_out',)), RenameAxes(old=('features_out',), new=('features',)), AddBias(bias=Parameter(value=<NamedArray float32(| features:128) ≈0.0 ±0.0 [≥0.0, ≤0.0] zero:128 (wrapping jax.Array)>, name='Affine_2.AddBias.bias'), new_axis_names=())],
    ),
    Elementwise(fn=jax.nn.relu),
    Affine( # Sequential
      sublayers=[Linear(weights=Parameter(value=<NamedArray float32(| features:128, features_out:64) ≈0.00012 ±0.1 [≥-0.18, ≤0.18] nonzero:8_192 (wrapping jax.Array)>, name='Affine_3.Linear.weights'), in_axis_names=('features',), out_axis_names=('features_out',)), RenameAxes(old=('features_out',), new=('features',)), AddBias(bias=Parameter(value=<NamedArray float32(| features:64) ≈0.0 ±0.0 [≥0.0, ≤0.0] zero:64 (wrapping jax.Array)>, name='Affine_3.AddBias.bias'), new_axis_names=())],
    ),
  ],
)

This printout is for the most part executable Python code, specifying the types and fields for each value. However, the individual types may not be in scope, and some values (like `jax.Array`) cannot be directly rebuilt from their `repr`.

You can fix this by running treescope in "roundtrip mode", which
- adds qualified names to all types
- disables non-roundtrippable summaries for special-cased types (like `NamedArray`)
- wraps any non-rountrippable type (like `jax.Array` or any type treescope doesn't know how to render) with a weak reference, enabling it to be copy-pasted within the current interpreter session (as long as the original object isn't garbage collected)

To toggle roundtrip mode, click on any output of treescope and press the "**`r`**" key. (Try it above!) Alternatively, pass `roundtrip_mode=True` to the renderer:

In [20]:
pz.ts.display(mlp, roundtrip_mode=True)

In roundtrip mode, as long as you've imported the necessary top-level modules, you should be able to select any part of the output, copy it, and paste it into another cell to rebuild an equivalent subtree to the one you copied.

### Function reflection and canonical aliases

Treescope has support for rendering useful information about functions and closures. The repr for functions isn't always very helpful, especially if wrapped by JAX:

In [21]:
repr(jax.nn.relu)

'<jax._src.custom_derivatives.custom_jvp object at 0x7a0f14a774c0>'

Treescope tries to figure out where functions, function-like objects, and other constants are defined, and uses that to summarize them when collapsed. This works for ordinary function definitions defined anywhere and also for function-like objects in the JAX public API (see `well_known_aliases.py`)

In [22]:
jax.nn.relu

For ordinary functions, it can even identify the file where the function was defined:

In [23]:
jnp.sum

<function jax._src.numpy.reductions.sum(a: 'ArrayLike', axis: 'Axis' = None, dtype: 'DTypeLike | None' = None, out: 'None' = None, keepdims: 'bool' = False, initial: 'ArrayLike | None' = None, where: 'ArrayLike | None' = None, promote_integers: 'bool' = True) -> 'Array'>

This works even for locally-defined notebook functions:

In [24]:
def my_function():
  print("hello world!")

In [25]:
my_function

<function __main__.my_function()>

### Embedded NDArray visualizer (`arrayviz`) and customizable figure inlining

Treescope includes a custom interactive NDArray visualizer designed to visualize the elements of high-dimensional arrays:

In [26]:
arr = (
    np.linspace(-10, 10, 20)
    * np.linspace(-10, 10, 15)[:, np.newaxis]
    * np.linspace(-1, 1, 5)[:, np.newaxis, np.newaxis]
)
pz.ts.render_array(arr)

ArrayvizRendering(html_src='<div id="arrayviz6f7eb9baa8864b069ea3c21e60bbbe32" class="arrayviz_container"><span class="loading_message">Rendering array...</span></div><template class="treescope_run_soon"><script> arrayviz.buildArrayvizFigure({"destinationId": "arrayviz6f7eb9baa8864b069ea3c21e60bbbe32", "info": "Dynamic colormap (click to adjust). Hover/click for array data.", "arrayBase64": "AADIwofyssIN5Z3ClNeIwjaUZ8JDeT3CUV4Twr2G0sGvoXzBymuowMprqECvoXxBvYbSQVFeE0JDeT1CNpRnQpTXiEIN5Z1Ch/KyQgAAyEK3bavCKmKZwp5Wh8IjlmrCCn9GwvFnIsKvofzBfXO0wZaKWMFkXJDAZFyQQJaKWEF9c7RBr6H8QfFnIkIKf0ZCI5ZqQp5Wh0IqYplCt22rQm7bjsKco3/CXJBhwh19Q8LdaSXCnlYHwr2G0sE+YJbBfXM0wfyZcMD8mXBAfXM0QT5glkG9htJBnlYHQt1pJUIdfUNCXJBhQpyjf0Ju245CSZJkwuOCTMJ9czTCF2QcwrFUBMKWitjBymuowfyZcMFkXBDBMHtAwDB7QEBkXBBB/JlwQcprqEGWithBsVQEQhdkHEJ9czRC44JMQkmSZEK3bSvCKmIZwp5WB8IjlurBCn/GwfFnosGvoXzBfXM0wZaK2MBkXBDAZFwQQJaK2EB9czRBr6F8QfFnokEKf8ZBI5bqQZ5WB0IqYhlCt20rQkmS5MHjgszBfXO0wRdknMGxVITBlopYwcprKMH8mfDAZFyQwDB7wL8we8A/ZFyQQPyZ8EDKayhBlopYQbFUhEEXZJxBfXO0QeOCzEFJkuRBSZJkweOCTMF9czTBF2QcwbFUBMGWitjAymuowPyZcMBkXBDAMHtAvzB7QD9kXBBA/JlwQMprqECWithAsVQEQRdkHEF9czRB44JMQUmSZEEAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAgAAAAIAAAACAAAAAgAAAAIAAAACAAAAAgAAAAIAAAACAAAAAgEmSZEHjgkxBfXM0QRdkHEGxVARBlorYQMprqED8mXBAZFwQQDB7QD8we0C/ZFwQwPyZcMDKa6jAlorYwLFUBMEXZBzBfXM0weOCTMFJkmTBSZLkQeOCzEF9c7RBF2ScQbFUhEGWilhBymsoQfyZ8EBkXJBAMHvAPzB7wL9kXJDA/JnwwMprKMGWiljBsVSEwRdknMF9c7TB44LMwUmS5MG3bStCKmIZQp5WB0IjlupBCn/GQfFnokGvoXxBfXM0QZaK2EBkXBBAZFwQwJaK2MB9czTBr6F8wfFnosEKf8bBI5bqwZ5WB8IqYhnCt20rwkmSZELjgkxCfXM0QhdkHEKxVARClorYQcprqEH8mXBBZFwQQTB7QEAwe0DAZFwQwfyZcMHKa6jBlorYwbFUBMIXZBzCfXM0wuOCTMJJkmTCbtuOQpyjf0JckGFCHX1DQt1pJUKeVgdCvYbSQT5glkF9czRB/JlwQPyZcMB9czTBPmCWwb2G0sGeVgfC3Wklwh19Q8JckGHCnKN/wm7bjsK3batCKmKZQp5Wh0IjlmpCCn9GQvFnIkKvofxBfXO0QZaKWEFkXJBAZFyQwJaKWMF9c7TBr6H8wfFnIsIKf0bCI5Zqwp5Wh8IqYpnCt22rwgAAyEKH8rJCDeWdQpTXiEI2lGdCQ3k9QlFeE0K9htJBr6F8QcprqEDKa6jAr6F8wb2G0sFRXhPCQ3k9wjaUZ8KU14jCDeWdwofyssIAAMjCAABIwofyMsIN5R3ClNcIwjaU58FDeb3BUV6Twb2GUsGvofzAymsowMprKECvofxAvYZSQVFek0FDeb1BNpTnQZTXCEIN5R1Ch/IyQgAASEK3bSvCKmIZwp5WB8IjlurBCn/GwfFnosGvoXzBfXM0wZaK2MBkXBDAZFwQQJaK2EB9czRBr6F8QfFnokEKf8ZBI5bqQZ5WB0IqYhlCt20rQm7bDsKco//BXJDhwR19w8HdaaXBnlaHwb2GUsE+YBbBfXO0wPyZ8L/8mfA/fXO0QD5gFkG9hlJBnlaHQd1ppUEdfcNBXJDhQZyj/0Fu2w5CSZLkweOCzMF9c7TBF2ScwbFUhMGWiljBymsowfyZ8MBkXJDAMHvAvzB7wD9kXJBA/JnwQMprKEGWilhBsVSEQRdknEF9c7RB44LMQUmS5EG3bavBKmKZwZ5Wh8EjlmrBCn9GwfFnIsGvofzAfXO0wJaKWMBkXJC/ZFyQP5aKWEB9c7RAr6H8QPFnIkEKf0ZBI5ZqQZ5Wh0EqYplBt22rQUmSZMHjgkzBfXM0wRdkHMGxVATBlorYwMprqMD8mXDAZFwQwDB7QL8we0A/ZFwQQPyZcEDKa6hAlorYQLFUBEEXZBxBfXM0QeOCTEFJkmRBSZLkwOOCzMB9c7TAF2ScwLFUhMCWiljAymsowPyZ8L9kXJC/MHvAvjB7wD5kXJA//JnwP8prKECWilhAsVSEQBdknEB9c7RA44LMQEmS5EAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAgAAAAIAAAACAAAAAgAAAAIAAAACAAAAAgAAAAIAAAACAAAAAgEmS5EDjgsxAfXO0QBdknECxVIRAlopYQMprKED8mfA/ZFyQPzB7wD4we8C+ZFyQv/yZ8L/KayjAlopYwLFUhMAXZJzAfXO0wOOCzMBJkuTASZJkQeOCTEF9czRBF2QcQbFUBEGWithAymuoQPyZcEBkXBBAMHtAPzB7QL9kXBDA/JlwwMprqMCWitjAsVQEwRdkHMF9czTB44JMwUmSZMG3batBKmKZQZ5Wh0EjlmpBCn9GQfFnIkGvofxAfXO0QJaKWEBkXJA/ZFyQv5aKWMB9c7TAr6H8wPFnIsEKf0bBI5ZqwZ5Wh8EqYpnBt22rwUmS5EHjgsxBfXO0QRdknEGxVIRBlopYQcprKEH8mfBAZFyQQDB7wD8we8C/ZFyQwPyZ8MDKayjBlopYwbFUhMEXZJzBfXO0weOCzMFJkuTBbtsOQpyj/0FckOFBHX3DQd1ppUGeVodBvYZSQT5gFkF9c7RA/JnwP/yZ8L99c7TAPmAWwb2GUsGeVofB3WmlwR19w8FckOHBnKP/wW7bDsK3bStCKmIZQp5WB0IjlupBCn/GQfFnokGvoXxBfXM0QZaK2EBkXBBAZFwQwJaK2MB9czTBr6F8wfFnosEKf8bBI5bqwZ5WB8IqYhnCt20rwgAASEKH8jJCDeUdQpTXCEI2lOdBQ3m9QVFek0G9hlJBr6H8QMprKEDKayjAr6H8wL2GUsFRXpPBQ3m9wTaU58GU1wjCDeUdwofyMsIAAEjCAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAIAAAACAAAAAgAAAAIAAAACAAAAAgAAAAIAAAACAAAAAgAAAAIAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAgAAAAIAAAACAAAAAgAAAAIAAAACAAAAAgAAAAIAAAACAAAAAgAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAACAAAAAgAAAAIAAAACAAAAAgAAAAIAAAACAAAAAgAAAAIAAAACAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAIAAAACAAAAAgAAAAIAAAACAAAAAg

It's integrated with the rest of treescope, making it possible to directly visualize entire nested containers of arrays at once. (Large arrays get automatically truncated along one or more axes to keep the visualization relatively small.)

In [27]:
with pz.ts.active_autovisualizer.set_scoped(pz.ts.ArrayAutovisualizer()):
  pz.ts.display((lambda root: root.sublayers[6].sublayers[0].weights.value)(mlp))

In [28]:
with pz.ts.active_autovisualizer.set_scoped(pz.ts.ArrayAutovisualizer()):
  # Visualizations with many items get collapsed by default.
  # Click to expand them.
  pz.ts.display(mlp)

If you want more control over how arrays and other objects are visualized, you can write your own visualization function and configure treescope to use it:

In [29]:
import plotly.express as px

def visualize_with_histograms(value, path):
  if isinstance(value, (np.ndarray, jax.Array)):
    # You can use any rich display object, for instance a plotly figure:
    return pz.ts.IPythonVisualization(
        px.histogram(
            value.flatten(),
            width=400, height=200
        ).update_layout(
            margin=dict(l=20, r=20, t=20, b=20)
        )
    )

In [30]:
with pz.ts.active_autovisualizer.set_scoped(visualize_with_histograms):
  pz.ts.display((lambda root: root.sublayers[6].sublayers[0].weights.value)(mlp))

You can use the `%%autovisualize` IPython magic to enable automatic visualization in a cell:

In [31]:
pz.ts.register_autovisualize_magic()

In [32]:
%%autovisualize
(lambda root: root.sublayers[6].sublayers[0].sublayers[0].weights.value)(mlp)

AttributeError: 'Linear' object has no attribute 'sublayers'

See the separate [array visualization tutorial](treescope_arrayviz.ipynb) for more info on how to visualize arrays and customize layouts!

## Where you can use treescope

### In IPython / Colab

Treescope works great in IPython and Colab notebooks, and is designed as a drop-in replacement for the IPython pretty-printer.

We've already done it above, but you can configure treescope as the default IPython formatter by calling

In [33]:
pz.ts.register_as_default()

or manually display specific objects with

In [34]:
pz.ts.display(["some object"])

There's also a helper function to show rich objects with syntax similar to Python's `print`:

In [35]:
pz.show("A value:", ["some object"])

TreescopeRenderingFigure(child=PreventWordWrap(child=Siblings(children=(Text(text='A value:'), Text(text=' '), WithDynamicSharedPip(child=FoldableTreeNodeImpl(contents=Siblings(children=(StringCopyButton(copy_string='(lambda root: root)', annotation='Copy path: '), IndentedChildren(children=[Siblings(children=(HyperlinkTarget(child=StringLiteralColor(child=Text(text="'some object'")), keypath=(SequenceKey(idx=0),)), FoldCondition(collapsed=EmptyPart(), expanded=Text(text=',')), FoldCondition(collapsed=EmptyPart(), expanded=Siblings(children=(StringCopyButton(copy_string='(lambda root: root[0])', annotation='Copy path: '),)))))]), Text(text=']'))), label=HyperlinkTarget(child=Text(text='['), keypath=()), expand_state=<ExpandState.COLLAPSED: 'collapsed'>), node_id=134166324290176, seen_more_than_once=set())))))

If you register treescope as the default IPython formatter, you can also just do

In [36]:
["some object"]

['some object']

### In the IPython / Colab debugger

It's actually also possible to use treescope inside the Colab debugger or IPython's `ipdb`. This isn't specific to treescope, and it may be somewhat fragile, but it's possible to monkey-patch `pdb` so that it uses `IPython.display.display` to display values that are output to the console. If you've registered treescope with IPython, that means you get access to all of it's formatting while inspecting stack frames!

There's an experimental wrapper that sets this up for you:

In [37]:
from penzai.toolshed import patch_ipdb
patch_ipdb.patch_ipdb()

Try running this and dropping into the debugger:

In [38]:
def my_function(some_input):
  assert some_input is None

# # Uncomment me:
# my_function({"a": 1, "b": np.arange(1000)})

### In ordinary Python for offline viewing

Treescope can render directly to static HTML, without requiring any dynamic communication between the Python kernel and the HTML renderer. This means you can directly save the output of a treescope rendering to an HTML file, and open it later to view whatever was formatted.

In [39]:
with pz.ts.active_autovisualizer.set_scoped(pz.ts.ArrayAutovisualizer()):
  contents = pz.ts.render_to_html(mlp, roundtrip_mode=True)

with open("/tmp/treescope_output.html", "w") as f:
  f.write(contents)

# Uncomment to download the file:
# import google.colab.files
# google.colab.files.download("/tmp/treescope_output.html")

## Things treescope can render

Treescope has support for a large number of common Python objects.

### Dicts, lists, tuples, and sets

In [40]:
[
    [(), (1,), (1, 2, 3)],
    {"foo": "bar", "baz": "qux"},
    {(1,2,3):(4,5,6), (7,8,9):(10,11,12)},
    {"a", "b", "c", "d"}
]

[[(), (1,), (1, 2, 3)],
 {'foo': 'bar', 'baz': 'qux'},
 {(1, 2, 3): (4, 5, 6), (7, 8, 9): (10, 11, 12)},
 {'a', 'b', 'c', 'd'}]

### Builtins and literals
(with special handling for multiline strings)

In [41]:
[
    [1, 2, 3, 4],
    ["a", "b", "c", "d"],
    [True, False, None, NotImplemented, Ellipsis],
    ["a\n  multiline\n    string"]
]

[[1, 2, 3, 4],
 ['a', 'b', 'c', 'd'],
 [True, False, None, NotImplemented, Ellipsis],
 ['a\n  multiline\n    string']]

### Dataclasses and namedtuples

In [42]:
class Foo(typing.NamedTuple):
  a: int
  b: str

Foo(a=1, b="bar")

Foo(a=1, b='bar')

In [43]:
@dataclasses.dataclass(frozen=True)
class Bar:
  c: str
  d: int
  some_list: list = dataclasses.field(default_factory=list)

IPython.display.display(Bar(c="bar", d=2))

Bar(c='bar', d=2, some_list=[])

In roundtrip mode, treescope will even help you rebuild dataclasses with weird `__init__` methods:

In [44]:
@dataclasses.dataclass
class WeirdInitClass:
  foo: int

  def __init__(self, half_foo: int):
    self.foo = 2 * half_foo

# This shows as WeirdInitClass(foo=4):
pz.ts.display(WeirdInitClass(2))

# But in roundtrip mode (explicit or after pressing `r`), it shows as
#   pz.dataclass_from_attributes(WeirdInitClass, foo=4)
# which bypasses __init__ and rebuilds the dataclass's attributes directly,
# since __init__ doesn't take `foo` as an argument.
pz.ts.display(WeirdInitClass(2), roundtrip_mode=True)

### NDArrays and NamedArrays
Treescope summarizes the shape, mean, standard deviation, bounds, and number of special values in any arrays. It also supports automatic visualization (as described above).

In [45]:
[
    jnp.arange(1000),
    np.array([[np.nan] * 100, [0] * 50 + [1] * 50]),
    pz.nx.arange("foo", 10) * pz.nx.arange("bar", 15),
]

[# jax.Array int32(1000,) [≥0, ≤999] zero:1 nonzero:999
   Array([  0,   1,   2, ..., 997, 998, 999], dtype=int32)
 ,
 array([[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan, nan, nan, nan, nan, nan, nan, nan],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,
          1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
          1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
          1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
          1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.]]),
 <NamedArray int32(| foo:10, bar:15) [≥0, ≤126] zero:24 nonzero:126 (wrapping jax.Array)>]

In [46]:
%%autovisualize
[
    jnp.arange(1000),
    np.array([[np.nan] * 100, [0] * 50 + [1] * 50]),
    pz.nx.arange("foo", 10) * pz.nx.arange("bar", 15),
]

[<jax.Array int32(1000,) [≥0, ≤999] zero:1 nonzero:999
   <Arrayviz rendering>
 >,
 array([[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan, nan, nan, nan, nan, nan, nan, nan],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,
          1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
          1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
          1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
          1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.]]),
 <NamedArray int32(| foo:10, bar:15) [≥0, ≤126] zero:24 nonzero:126 (wrapping jax.Array)
   <Arrayviz rendering>
 >]

When used in IPython, Treescope will try to render the tree structure first and then insert array visualizations later. This can make visualization faster and can sometimes let you see the shape of JAX arrays before JAX has finished computing their values.

### `pz.Struct`s, layers, and models

`pz.Struct` and `pz.Layer` types are dataclasses, and they render similarly to ordinary dataclasses, but with a few extra features:

- Layers are color-coded by type, and other blocks can opt-in to color-coding by defining `treescope_color`.
- Layers also print a summary of their input and output structures, if known.
- Complex models built using `penzai.data_effects` have extra annotations for tracking the effects and their handlers.

In [47]:
initialized_mlp = pz.nn.initialize_parameters(
  simple_mlp.MLP.from_config(
    feature_sizes=[32, 64, 64, 16]
  ), jax.random.key(0)
)
initialized_mlp

MLP( # Sequential
  sublayers=[
    Affine( # Sequential
      sublayers=[
        Linear(weights=Parameter(value=<NamedArray float32(| features:32, features_out:64) ≈-0.0026 ±0.15 [≥-0.25, ≤0.25] nonzero:2_048 (wrapping jax.Array)>, name='Affine_0.Linear.weights'), in_axis_names=('features',), out_axis_names=('features_out',)),
        RenameAxes(old=('features_out',), new=('features',)),
        AddBias(bias=Parameter(value=<NamedArray float32(| features:64) ≈0.0 ±0.0 [≥0.0, ≤0.0] zero:64 (wrapping jax.Array)>, name='Affine_0.AddBias.bias'), new_axis_names=()),
      ],
    ),
    Elementwise(fn=jax.nn.relu),
    Affine( # Sequential
      sublayers=[Linear(weights=Parameter(value=<NamedArray float32(| features:64, features_out:64) ≈-0.0014 ±0.13 [≥-0.22, ≤0.22] nonzero:4_096 (wrapping jax.Array)>, name='Affine_1.Linear.weights'), in_axis_names=('features',), out_axis_names=('features_out',)), RenameAxes(old=('features_out',), new=('features',)), AddBias(bias=Parameter(value=<NamedArray float32(| features:64) ≈0.0 ±0.0 [≥0.0, ≤0.0] zero:64 (wrapping jax.Array)>, name='Affine_1.AddBias.bias'), new_axis_names=())],
    ),
    Elementwise(fn=jax.nn.relu),
    Affine( # Sequential
      sublayers=[Linear(weights=Parameter(value=<NamedArray float32(| features:64, features_out:16) ≈-0.004 ±0.16 [≥-0.27, ≤0.27] nonzero:1_024 (wrapping jax.Array)>, name='Affine_2.Linear.weights'), in_axis_names=('features',), out_axis_names=('features_out',)), RenameAxes(old=('features_out',), new=('features',)), AddBias(bias=Parameter(value=<NamedArray float32(| features:16) ≈0.0 ±0.0 [≥0.0, ≤0.0] zero:16 (wrapping jax.Array)>, name='Affine_2.AddBias.bias'), new_axis_names=())],
    ),
  ],
)

### Functions
(As discussed in the features section)

In [48]:
[
    jnp.sum,
    dataclasses.dataclass,
    lambda x: x + 2,
    jax.vmap(lambda x: x),
]

[<function jax._src.numpy.reductions.sum(a: 'ArrayLike', axis: 'Axis' = None, dtype: 'DTypeLike | None' = None, out: 'None' = None, keepdims: 'bool' = False, initial: 'ArrayLike | None' = None, where: 'ArrayLike | None' = None, promote_integers: 'bool' = True) -> 'Array'>,
 <function dataclasses.dataclass(cls=None, /, *, init=True, repr=True, eq=True, order=False, unsafe_hash=False, frozen=False, match_args=True, kw_only=False, slots=False)>,
 <function __main__.<lambda>(x)>,
 <function __main__.<lambda>(x)>]

### Arbitrary PyTree types
Treescope uses a fallback rendering strategy to show the children of any PyTree type registered with JAX, even if it isn't usually supported by treescope.

In [49]:
jax.tree_util.Partial(lambda x, y, z: x + y, 10, y=100)

Partial(<function <lambda> at 0x7a05ac1fdfc0>, 10, y=100)

### Partial support: Repeated Python object references
Treescope will warn you if it sees multiple references to the same mutable object, since that can cause unexpected behavior. (In this case, copying the output won't copy the shared reference structure.)

In [50]:
my_shared_list = []

{
    "foo": my_shared_list,
    "bar": my_shared_list,
    "baz": [1, 2, my_shared_list]
}

{'foo': [], 'bar': [], 'baz': [1, 2, []]}